In [2]:
ANTHROPIC_API_KEY=''

In [14]:
import anthropic

client = anthropic.Anthropic(api_key = ANTHROPIC_API_KEY)

def get_response(text):
    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=4096,
        temperature=0,
        system="You are an expert medical diagnostic engine.",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": text
                    }
                ]
            }
        ]
    )
    return message.content

In [15]:
get_response('How do you diagnose sepsis?')

[TextBlock(text="Diagnosing sepsis involves a combination of clinical assessment, physical examination, and laboratory tests. Here's an overview of the diagnostic process:\n\n1. Clinical Assessment:\n   - Evaluate patient history and risk factors\n   - Look for signs of infection\n   - Check for symptoms like fever, chills, rapid breathing, and confusion\n\n2. Physical Examination:\n   - Measure vital signs (temperature, heart rate, respiratory rate, blood pressure)\n   - Check for signs of organ dysfunction\n\n3. Laboratory Tests:\n   - Complete Blood Count (CBC): Look for elevated white blood cell count\n   - Blood cultures: To identify the causative organism\n   - Lactate levels: Elevated lactate can indicate tissue hypoperfusion\n   - Procalcitonin: Can be elevated in bacterial infections\n   - C-reactive protein (CRP): Marker of inflammation\n   - Organ function tests: Kidney function (creatinine), liver function tests, etc.\n\n4. Imaging Studies:\n   - Chest X-ray, CT scans, or u

In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm

result_df = pd.read_csv('PaLM2input.csv', index_col=0)

result_df['GPT_input'] = result_df['GPT_input'].str.replace('minimum', 'min')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maximum', 'max')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('average', 'avg')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maxntprobnp', 'max ntprobnp')

result_df['GPT-Diagnoses'] = np.nan
result_df['GPT-Eval'] = np.nan

In [17]:
prompt_no_example = '''Suggest as many potential diagnoses as possible from patient data.
In addition, include previously diagnosed conditions and information about patient's medical history (if any). 
Give exact numbers and/or text quotes from the data that made you think of each of the diagnoses \
and, if necessary, give further tests that could confirm the diagnosis. 
Once you're done, suggest further, more complex diseases that may be ongoing based on the existing diagnoses you already made. 
Use the International Classification of Disease (ICD) naming standard for reporting the diagnoses, but you don't have to specify the codes.
Use the same spacing as punctuation in your answer as in the example below.

Before finalizing your answer check if you haven't missed any abnormal data points and hence any diagnoses that could be made \
based on them. If you did, add them to your list of diagnoses.'''

example = '''

For example, if the patient data mentions:

"Blood report: 
min glucose: 103, max glucose: 278, avg glucose: 156.5, max inr: 2.1, max pt: 22.4, max ptt: 150, \
avg wbc: 13.8, max wbc: 14.1, max lactate: 5.9, max bun: 101, max creatinine: 5.8, avg bun: 38.15, avg creatinine: 2.78
Blood gas report: 
3 hours after admission the blood gas results from venous blood are: ph: 7.2
Imaging report: 
Status post left total shoulder replacement
Chest X-Ray Possible small right pleural effusion and Mild, bibasilar atelectasis.. \
Lung volumes have slightly increased but areas of atelectasis are seen at both the left and the right lung bases
Microbiology tests: 
24 hours after admission the microbiology culture test MRSA SCREEN obtained via MRSA SCREEN \
identified POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS
Vitalsigns data from ICU: 
max temperature: 38, min peripheral oxygen saturation: 70, max respiration rate: 29"

then your answer may be: 

1: Methicillin resistant Staphylococcus aureus infection, site unspecified
Foundational data: Microbiology culture test identifying "POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS"

2: Atelectasis 
Foundational data from Chest X-Ray: "Mild, bibasilar atelectasis.. \
Lung volumes have slightly increased but areas of atelectasis are seen at both the left and the right lung bases"

3: Pleural effusion, not elsewhere classified
Foundational data from Chest X-Ray: "Possible small right pleural effusion."
Further tests: Thoracentesis, CT chest

4: Acidosis
Foundational data: "ph: 7.2"
Further tests: Urine pH, Anion Gap

5: Lactic acidosis
Foundational data: "max lactate: 5.9"

6: Acquired coagulation factor deficiency
Foundational data: "max inr: 2.1, max pt: 22.4, max ptt: 150"
Further tests: Antiphospholipid Antibodies (APL), Protein C, Protein S, Antithrombin III, Factor V Leiden, Fibrinogen test

7: Hyperglycemia, unspecified
Foundational data: "max glucose: 278, avg glucose: 156.5".
Further tests: Hemoglobin A1c (HbA1c) test

8: Hypoxemia
Foundational data: "min peripheral oxygen saturation: 70"
Further tests: Measure PaO2 in blood

9: Leukocytosis
Foundational data: "max wbc: 14.1, avg wbc: 13.8". The patient's white blood cell count is consistently elevated which may suggest an ongoing inflammatory response or infection.
Further tests: Infection markers such as CRP or PCT, Assessment of symptoms like fever, inflammation or fatigue. 

10. Unspecified acute kidney failure:
Foundational data: "max bun: 101, max creatinine: 5.8, avg bun: 38.15, avg creatinine: 2.78"
Further tests: Urine output measurements for oliguria, ultrasound to rule out obstruction

11. Presence of left artificial shoulder joint
Foundational data: The imaging report mentions: "Status post left total shoulder replacement"

Further diseases based on these diagnoses (continued the indexing from the previous number in the list):

12: Unspecified septicemia
Foundational data: positive MRSA screen, systemic inflammatory response: "max respiration rate: 29", "max temperature: 38", leukocytosis
Further tests: HR, BP, wound culture, respiratory excretion tests

13: Septic shock
Foundational data: Septicemia with acidosis and lactic acidosis may suggest septic shock
Further tests: patient examination (low BP, mental disorientation, nausea, pale skin may confirm the finding)

14: Acute respiratory failure, with hypoxia or hypercapnia
Foundational data: hypoxemia and the presence of atelectasis
Further tests: Clinical symptoms (severe shortness of breath, rapid breathing, and confusion), \
arterial blood gas measurements showing hypoxia or hypercapnia

15: Type 2 diabetes mellitus with diabetic chronic kidney disease
Foundational data: Hyperglycemia and kidney failure
Further tests: urine test, hemoglobin (A1C) test, GFR, BP, physical examination (swelling, nausea, weakness, eye disease)'''

patient_data = '''

Patient data:\n'''

In [32]:
repeat = []
todo = [161] + list(range(265,1000))
for i in tqdm(range(766,1000)):
    try:
        response = get_response(
            prompt_no_example + example + patient_data + str(result_df.iloc[i]['GPT_input']),
        )
        hadm_id = result_df.index[i]
        result_df.loc[hadm_id, 'GPT-Diagnoses'] = response
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        repeat.append(i)
        print(e)
        


100%|██████████████████████████████████████████████████████████████████████████████| 234/234 [1:01:27<00:00, 15.76s/it]


In [33]:
result_df.to_csv('Claude_result.csv')

In [21]:
result_df.iloc[0,:]['GPT-Diagnoses']

TextBlock(text='Based on the provided patient data, I can suggest the following potential diagnoses:\n\n1: Leukocytosis\nFoundational data: "min wbc: 11, max wbc: 19.5, avg wbc: 15.08". The patient\'s white blood cell count is consistently elevated.\nFurther tests: Differential white blood cell count, CRP, PCT\n\n2: Hyperglycemia, unspecified\nFoundational data: "min glucose: 118, max glucose: 141, avg glucose: 128"\nFurther tests: Hemoglobin A1c (HbA1c) test\n\n3: Mild anemia, unspecified\nFoundational data: "min hemoglobin (g/dl): 11.4, min hematocrit: 33.3"\nFurther tests: Iron studies, vitamin B12, folate levels\n\n4: Thrombocytopenia, unspecified\nFoundational data: "min platelet: 127"\nFurther tests: Peripheral blood smear, bone marrow biopsy if persistent\n\n5: Atelectasis\nFoundational data: "There are minimal left atelectasis", "Bibasal areas of atelectasis are present"\nFurther tests: Chest CT scan\n\n6: Cardiomegaly\nFoundational data: "Heart size is enlarged, stable"\nFurth